In [1]:
import numpy as np
import cvxpy as cp
from utils import parse_m_file, compute_PTDF

np.set_printoptions(precision=3, suppress=True, floatmode='fixed')


In [2]:

# Parameters
num_scenarios = 50
perturbation_scale = 0.75
baseMVA = 100

# Load data
bus_data, gen_data, branch_data, gencost_data = parse_m_file('pglib_opf_case14_ieee.m')
num_buses = bus_data.shape[0]
num_generators = gen_data.shape[0]
num_lines = branch_data.shape[0]
num_branches = branch_data.shape[0]

# Data Extraction
Pd_base = bus_data[:, 2]
Pg_min = gen_data[:, 9]
Pg_max = gen_data[:, 8]
Pg_max[0] = 400
Pg_max[1] = 100
cost_coeff_true = gencost_data[:, 5]

branch_data_congested = branch_data.copy()
branch_data_congested[:, 5] *= 0.7  # reduce limits by 30%

# PTDF = compute_PTDF(branch_data, bus_data)
PTDF = compute_PTDF(branch_data_congested, bus_data)


# Generator incidence matrix
gen_to_bus = np.zeros((num_buses, num_generators))
for i, gen_bus in enumerate(gen_data[:, 0].astype(int) - 1):
    gen_to_bus[gen_bus, i] = 1

# Storage for scenario data
Pg_scenarios = []
lambda_slack_scenarios = []
nu_max_scenarios = []
nu_min_scenarios = []
mu_min_scenarios = []
mu_max_scenarios = []
Pd_scenarios = []

scenario_count = 0
np.random.seed(42)

while scenario_count < num_scenarios:
    Pd_perturbed = Pd_base * (1 + np.random.uniform(-perturbation_scale, perturbation_scale, size=num_buses))
    #Pd_perturbed = np.random.randint(0, 100, size=Pd_base.shape) + np.random.randint(0, 100, size=Pd_base.shape)/100
    Pd_perturbed[0] = 0  # slack bus
    Pd_scenarios.append(Pd_perturbed)
    
    Pg = cp.Variable(num_generators)
    P_inj = gen_to_bus @ Pg - Pd_perturbed
    P_inj_reduced = P_inj[1:]

    constraints = [
        cp.sum(Pg) == np.sum(Pd_perturbed),
        Pg >= Pg_min,
        Pg <= Pg_max,
        -branch_data_congested[:, 5] <= PTDF @ P_inj_reduced,
        PTDF @ P_inj_reduced <= branch_data_congested[:, 5]
    ]

    objective = cp.Minimize(cp.sum(cp.multiply(cost_coeff_true, Pg)))
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)

    if prob.status == 'optimal':
        scenario_count += 1
        Pg_scenarios.append(Pg.value)
        Pd_scenarios.append(Pd_perturbed)
        lambda_slack_scenarios.append(constraints[0].dual_value)
        nu_min_scenarios.append(constraints[3].dual_value)
        nu_max_scenarios.append(constraints[4].dual_value)
        mu_min_scenarios.append(constraints[1].dual_value)
        mu_max_scenarios.append(constraints[2].dual_value)
        print(f"Scenario {scenario_count}/{num_scenarios} generated.")

Pg_scenarios = np.array(Pg_scenarios)
lambda_slack_scenarios = np.array(lambda_slack_scenarios)
nu_max_scenarios = np.array(nu_max_scenarios)
nu_min_scenarios = np.array(nu_min_scenarios)
mu_min_scenarios = np.array(mu_min_scenarios)
mu_max_scenarios = np.array(mu_max_scenarios)



Scenario 1/50 generated.
Scenario 2/50 generated.
Scenario 3/50 generated.
Scenario 4/50 generated.
Scenario 5/50 generated.
Scenario 6/50 generated.
Scenario 7/50 generated.
Scenario 8/50 generated.
Scenario 9/50 generated.
Scenario 10/50 generated.
Scenario 11/50 generated.
Scenario 12/50 generated.
Scenario 13/50 generated.
Scenario 14/50 generated.
Scenario 15/50 generated.
Scenario 16/50 generated.
Scenario 17/50 generated.
Scenario 18/50 generated.
Scenario 19/50 generated.
Scenario 20/50 generated.
Scenario 21/50 generated.
Scenario 22/50 generated.
Scenario 23/50 generated.
Scenario 24/50 generated.
Scenario 25/50 generated.
Scenario 26/50 generated.
Scenario 27/50 generated.
Scenario 28/50 generated.
Scenario 29/50 generated.
Scenario 30/50 generated.
Scenario 31/50 generated.
Scenario 32/50 generated.
Scenario 33/50 generated.
Scenario 34/50 generated.
Scenario 35/50 generated.
Scenario 36/50 generated.
Scenario 37/50 generated.
Scenario 38/50 generated.
Scenario 39/50 genera

In [3]:
print("lambda_slack_scenarios:\n", lambda_slack_scenarios)
print("Pg_scenarios:\n", Pg_scenarios)
print("Pd_scenarios:\n", Pd_scenarios)
print("nu_max_scenarios:\n", nu_max_scenarios)
print("nu_min_scenarios:\n", nu_min_scenarios)  

lambda_slack_scenarios:
 [-7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921
 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921
 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921
 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921
 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921]
Pg_scenarios:
 [[308.437   4.756  -0.000  -0.000   0.000]
 [204.140   0.000  -0.000  -0.000   0.000]
 [216.973   0.000  -0.000  -0.000   0.000]
 [289.907   0.000  -0.000  -0.000   0.000]
 [161.240   0.000  -0.000  -0.000   0.000]
 [186.555   0.000  -0.000  -0.000   0.000]
 [284.246  27.891  -0.000  -0.000   0.000]
 [170.197   0.000  -0.000  -0.000   0.000]
 [289.057  30.941  -0.000  -0.000   0.000]
 [223.186   0.000  -0.000  -0.000   0.000]
 [195.520   0.000  -0.000  -0.000   0.000]
 [291.201   0.000  -0.000  -0.000   0.000]
 [241.970   0.000  -0.000  -0.000   0.000]
 [237.142   0.000  -0.000  -0.000   0.000]
 [

In [6]:
# Inverse Optimization (using multiple scenarios)
c = cp.Variable(num_generators, nonneg=True)
loss = 0

Pg_inv = cp.Variable((num_scenarios, num_generators))
lambda_slack_inv = cp.Variable(num_scenarios)   
nu_min_inv = cp.Variable((num_scenarios, num_lines))
nu_max_inv = cp.Variable((num_scenarios, num_lines))
mu_min_inv = cp.Variable((num_scenarios, num_generators))
mu_max_inv = cp.Variable((num_scenarios, num_generators))

constraints_inv = []
stationarity = []
primal_feasibility = []
dual_feasibility = []
complementary = []

for t in range(num_scenarios):
    
    P_inj_inv = gen_to_bus @ Pg_inv[t] - Pd_scenarios[t]
    P_inj_reduced_inv = P_inj_inv[1:]
    flow_lines = PTDF @ P_inj_reduced_inv
    
    for gen in range(num_generators):
        # for line in range(num_lines):
            # Partial derivative of P_inj_reduced w.r.t Pg[i] is gen_to_bus[1:, i]
            # congestion_term_inv += (nu_max_inv[t][line] - nu_min_inv[t][line]) * PTDF[line] @ gen_to_bus[1:, gen]
        stationarity.append(
            c[gen] + lambda_slack_inv[t] + mu_max_inv[t][gen] - mu_max_inv[t][gen]      
                    + (nu_max_inv[t] + nu_min_inv[t]) @ PTDF @ gen_to_bus[1:, gen] == 0
        )

    # primal_feasibility.append(
    #     cp.sum(Pg_inv[t]) == np.sum(Pd_scenarios[t]))
    #     # Pg_inv[t] >= Pg_min,
    #     # Pg_inv[t] <= Pg_max,
    #     # -branch_data[:, 5] <= PTDF @ P_inj_reduced_inv,
    #     # PTDF @ P_inj_reduced_inv <= branch_data[:, 5]        Replace it with the dependent dual variables
    
    primal_feasibility += [
        cp.sum(Pg_inv) == np.sum(Pd_scenarios[t]),
        Pg_inv >= Pg_min,
        Pg_inv <= Pg_max,
        -branch_data[:, 5] <= flow_lines,
        flow_lines <= branch_data[:, 5]
    ]
    
    dual_feasibility += [
        mu_min_inv >= 0,
        mu_max_inv >= 0,
        nu_min_inv >= 0,
        nu_max_inv >= 0
    ]
    # dual_feasibility.append(mu_min_inv[t] >= 0)
    # dual_feasibility.append(mu_max_inv[t] >= 0)
    # dual_feasibility.append(nu_min_inv[t] >= 0)
    # dual_feasibility.append(nu_max_inv[t] >= 0)
    
    
    for i in range(num_branches):
        # nu_min * (- branch_data_congested[:, 5] - PTDF @ P_inj_reduced) == 0
        if nu_min_scenarios[t][i] != 0:
            complementary.append(-branch_data_congested[i, 5] == flow_lines[i])
            dual_feasibility.append(nu_max_inv[t][i] == 0)
        # nu_max * (PTDF @ P_inj_reduced - branch_data_congested[:, 5]) == 0 
        elif nu_max_scenarios[t][i] != 0:
            complementary.append(flow_lines[i] == branch_data_congested[i, 5])
            dual_feasibility.append(nu_min_inv[t][i] == 0) # check this
        else:     
            complementary.append(nu_min_inv[t][i] == 0)
            complementary.append(nu_max_inv[t][i] == 0)
           
    
    for i in range(num_generators):
        # mu_max * (Pg - Pg_max) == 0
        if Pg_scenarios[t][i] == Pg_max[i]:
            dual_feasibility.append(mu_max_inv[t][i] >= 0)
            dual_feasibility.append(mu_min_inv[t][i] == 0)
        # mu_min * (Pg_min - Pg) == 0
        elif Pg_scenarios[t][i] == Pg_min[i]:
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
            dual_feasibility.append(mu_max_inv[t][i] == 0)
        else:
            dual_feasibility.append(mu_max_inv[t][i] == 0)
            dual_feasibility.append(mu_min_inv[t][i] == 0)

constraints_inv += stationarity + primal_feasibility  + dual_feasibility + complementary

constraints_inv.append(c[2:5] == 0)

loss += cp.sum_squares(Pg_inv - Pg_scenarios)
loss += cp.sum_squares(lambda_slack_inv - lambda_slack_scenarios)
loss += cp.sum_squares(mu_min_inv - mu_min_scenarios)
loss += cp.sum_squares(mu_max_inv - mu_max_scenarios)
loss += cp.sum_squares(nu_min_inv - nu_min_scenarios)
loss += cp.sum_squares(nu_max_inv - nu_max_scenarios)


loss *= 1 / num_scenarios

inv_prob = cp.Problem(cp.Minimize(loss), constraints_inv)
inv_prob.solve(solver=cp.GUROBI, verbose=True, reoptimize=True)

if inv_prob.status == 'optimal':
    print("Inferred cost coefficients: ", c.value)
    print("Optimal cost: ", inv_prob.value)
else:
    print("Inverse problem not optimal: ", inv_prob.status)


                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Apr 28 07:23:57 AM: Your problem has 2805 variables, 154803 constraints, and 0 parameters.
(CVXPY) Apr 28 07:23:58 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 28 07:23:58 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 28 07:23:58 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 28 07:23:58 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 28 07:23:58 AM: Compiling problem (target solver=GURO

/Users/don_williams09/opt/miniconda3/envs/opt_env/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


(CVXPY) Apr 28 07:24:04 AM: Applying reduction GUROBI
(CVXPY) Apr 28 07:24:04 AM: Finished problem compilation (took 6.224e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Apr 28 07:24:04 AM: Invoking solver GUROBI  to obtain a solution.
Set parameter OutputFlag to value 1
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
QCPDual  1

Optimize a model with 157608 rows, 5605 columns and 186535 nonzeros
Model fingerprint: 0x5de30628
Model has 2800 quadratic objective terms
Coefficient s

In [ ]:
# Inverse Optimization (using multiple scenarios)
c = cp.Variable(num_generators, nonneg=True)
loss = 0

Pg_inv = cp.Variable((num_scenarios, num_generators))
lambda_slack_inv = cp.Variable(num_scenarios)   
nu_min_inv = cp.Variable((num_scenarios, num_lines))
nu_max_inv = cp.Variable((num_scenarios, num_lines))
mu_min_inv = cp.Variable((num_scenarios, num_generators))
mu_max_inv = cp.Variable((num_scenarios, num_generators))

constraints_inv = []
stationarity = []
primal_feasibility = []
dual_feasibility = []
complementary = []

for t in range(num_scenarios):
    
    P_inj_inv = gen_to_bus @ Pg_inv[t] - Pd_scenarios[t]
    P_inj_reduced_inv = P_inj_inv[1:]
    flow_lines = PTDF @ P_inj_reduced_inv
    
    for gen in range(num_generators):
        # for line in range(num_lines):
            # Partial derivative of P_inj_reduced w.r.t Pg[i] is gen_to_bus[1:, i]
            # congestion_term_inv += (nu_max_inv[t][line] - nu_min_inv[t][line]) * PTDF[line] @ gen_to_bus[1:, gen]
        stationarity.append(
            c[gen] + lambda_slack_inv[t] + mu_max_inv[t][gen] - mu_max_inv[t][gen]      
                    + (nu_max_inv[t] + nu_min_inv[t]) @ PTDF @ gen_to_bus[1:, gen] == 0
        )

    # primal_feasibility.append(
    #     cp.sum(Pg_inv[t]) == np.sum(Pd_scenarios[t]))
    #     # Pg_inv[t] >= Pg_min,
    #     # Pg_inv[t] <= Pg_max,
    #     # -branch_data[:, 5] <= PTDF @ P_inj_reduced_inv,
    #     # PTDF @ P_inj_reduced_inv <= branch_data[:, 5]        Replace it with the dependent dual variables
    
    primal_feasibility += [
        cp.sum(Pg_inv) == np.sum(Pd_scenarios[t]),
        Pg_inv >= Pg_min,
        Pg_inv <= Pg_max,
        -branch_data[:, 5] <= flow_lines,
        flow_lines <= branch_data[:, 5]
    ]
    
    dual_feasibility += [
        mu_min_inv >= 0,
        mu_max_inv >= 0,
        nu_min_inv >= 0,
        nu_max_inv >= 0
    ]
    # dual_feasibility.append(mu_min_inv[t] >= 0)
    # dual_feasibility.append(mu_max_inv[t] >= 0)
    # dual_feasibility.append(nu_min_inv[t] >= 0)
    # dual_feasibility.append(nu_max_inv[t] >= 0)
    
    
    for i in range(num_branches):
        # nu_min * (- branch_data_congested[:, 5] - PTDF @ P_inj_reduced) == 0
        if nu_min_scenarios[t][i] != 0:
            complementary.append(-branch_data_congested[i, 5] == flow_lines[i])
            dual_feasibility.append(nu_max_inv[t][i] == 0)
        # nu_max * (PTDF @ P_inj_reduced - branch_data_congested[:, 5]) == 0 
        elif nu_max_scenarios[t][i] != 0:
            complementary.append(flow_lines[i] == branch_data_congested[i, 5])
            dual_feasibility.append(nu_min_inv[t][i] == 0) # check this
        else:     
            complementary.append(nu_min_inv[t][i] == 0)
            complementary.append(nu_max_inv[t][i] == 0)
           
    
    for i in range(num_generators):
        # mu_max * (Pg - Pg_max) == 0
        if Pg_scenarios[t][i] == Pg_max[i]:
            dual_feasibility.append(mu_max_inv[t][i] >= 0)
            dual_feasibility.append(mu_min_inv[t][i] == 0)
        # mu_min * (Pg_min - Pg) == 0
        elif Pg_scenarios[t][i] == Pg_min[i]:
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
            dual_feasibility.append(mu_max_inv[t][i] == 0)
        else:
            dual_feasibility.append(mu_max_inv[t][i] == 0)
            dual_feasibility.append(mu_min_inv[t][i] == 0)

constraints_inv += stationarity + primal_feasibility  + dual_feasibility + complementary

constraints_inv.append(c[2:5] == 0)

loss += cp.sum_squares(Pg_inv - Pg_scenarios)
loss += cp.sum_squares(lambda_slack_inv - lambda_slack_scenarios)
loss += cp.sum_squares(mu_min_inv - mu_min_scenarios)
loss += cp.sum_squares(mu_max_inv - mu_max_scenarios)
loss += cp.sum_squares(nu_min_inv - nu_min_scenarios)
loss += cp.sum_squares(nu_max_inv - nu_max_scenarios)


loss *= 1 / num_scenarios

inv_prob = cp.Problem(cp.Minimize(loss), constraints_inv)
inv_prob.solve(solver=cp.GUROBI, verbose=True, reoptimize=True)

if inv_prob.status == 'optimal':
    print("Inferred cost coefficients: ", c.value)
    print("Optimal cost: ", inv_prob.value)
else:
    print("Inverse problem not optimal: ", inv_prob.status)


In [ ]:
# Inverse Optimization (using multiple scenarios)
c = cp.Variable(num_generators, nonneg=True)
loss = 0

Pg_inv = cp.Variable((num_scenarios, num_generators))
lambda_slack_inv = cp.Variable(num_scenarios)   
nu_min_inv = cp.Variable((num_scenarios, num_lines))
nu_max_inv = cp.Variable((num_scenarios, num_lines))
mu_min_inv = cp.Variable((num_scenarios, num_generators))
mu_max_inv = cp.Variable((num_scenarios, num_generators))

constraints_inv = []
stationarity = []
primal_feasibility = []
dual_feasibility = []
complementary = []

for t in range(num_scenarios):
    
    P_inj_inv = gen_to_bus @ Pg_inv[t] - Pd_scenarios[t]
    P_inj_reduced_inv = P_inj_inv[1:]
    flow_lines = PTDF @ P_inj_reduced_inv
    
    for gen in range(num_generators):
        # for line in range(num_lines):
            # Partial derivative of P_inj_reduced w.r.t Pg[i] is gen_to_bus[1:, i]
            # congestion_term_inv += (nu_max_inv[t][line] - nu_min_inv[t][line]) * PTDF[line] @ gen_to_bus[1:, gen]
        stationarity.append(
            c[gen] + lambda_slack_inv[t] + mu_max_inv[t][gen] - mu_max_inv[t][gen]      
                    + (nu_max_inv[t] + nu_min_inv[t]) @ PTDF @ gen_to_bus[1:, gen] == 0
        )

    primal_feasibility.append(
        cp.sum(Pg_inv[t]) == np.sum(Pd_scenarios[t]))
        # Pg_inv[t] >= Pg_min,
        # Pg_inv[t] <= Pg_max,
        # -branch_data[:, 5] <= PTDF @ P_inj_reduced_inv,
        # PTDF @ P_inj_reduced_inv <= branch_data[:, 5]        Replace it with the dependent dual variables
    
    
    # dual_feasibility.append(mu_min_inv[t] >= 0)
    # dual_feasibility.append(mu_max_inv[t] >= 0)
    # dual_feasibility.append(nu_min_inv[t] >= 0)
    # dual_feasibility.append(nu_max_inv[t] >= 0)
    
    
    for i in range(num_branches):
        if nu_min_scenarios[t][i] != 0:
            complementary.append(-branch_data_congested[i, 5] == flow_lines[i])
            dual_feasibility.append(nu_min_inv[t][i] >= 0)
        else:
            complementary.append(nu_min_inv[t][i] == 0)
            primal_feasibility.append(-branch_data_congested[i, 5] <= flow_lines[i])     
    for i in range(num_branches):
        if nu_max_scenarios[t][i] != 0:
            complementary.append(flow_lines[i] == branch_data_congested[i, 5])
            dual_feasibility.append(nu_max_inv[t][i] >= 0)
        else:
            complementary.append(nu_max_inv[t][i] == 0)
            primal_feasibility.append(flow_lines[i] <= branch_data_congested[i, 5])
            
    for i in range(num_generators):
        if mu_min_scenarios[t][i] != 0:
            complementary.append(Pg_inv[t][i] == Pg_min[i])
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
        else:
            complementary.append(mu_min_inv[t][i] == 0)
            primal_feasibility.append(Pg_inv[t][i] >= Pg_min[i])
    for i in range(num_generators):
        if mu_max_scenarios[t][i] != 0:
            complementary.append(Pg_inv[t][i] == Pg_max[i])
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
        else:
            complementary.append(mu_max_inv[t][i] == 0)
            primal_feasibility.append(Pg_inv[t][i] <= Pg_max[i])

    loss += cp.norm(Pg_inv[t] - Pg_scenarios[t], 2)**2
    loss += cp.norm(lambda_slack_inv[t] - lambda_slack_scenarios[t], 2)**2
    for i in range(num_generators):
        loss += cp.norm(mu_min_inv[t][i] - mu_min_scenarios[t][i], 2)**2
        loss += cp.norm(mu_max_inv[t][i] - mu_max_scenarios[t][i], 2)**2
    for i in range(num_lines):      
        loss += cp.norm(nu_max_inv[t][i] - nu_max_scenarios[t][i], 2)**2
        loss += cp.norm(nu_min_inv[t][i] - nu_min_scenarios[t][i], 2)**2

constraints_inv += stationarity + primal_feasibility  + dual_feasibility + complementary

constraints_inv.append(c[2:5] == 0)
loss *= 1 / num_scenarios

inv_prob = cp.Problem(cp.Minimize(loss), constraints_inv)
inv_prob.solve(solver=cp.MOSEK, verbose=True)

if inv_prob.status == 'optimal':
    print("Inferred cost coefficients: ", c.value)
else:
    print("Inverse problem not optimal: ", inv_prob.status)


In [ ]:
from utils import *

print("Inferred cost coefficients: ", c.value)
print("True cost coefficients: ", cost_coeff_true)

# Inverse Optimization Variables
print("Pg_inv:\n", Pg_inv.value)
print("lambda_slack_inv:\n", lambda_slack_inv.value)
print("nu_max_inv:\n", nu_max_inv.value)
print("nu_min_inv:\n", nu_min_inv.value)
print("mu_min_inv:\n", mu_min_inv.value)
print("mu_max_inv:\n", mu_max_inv.value)